In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
from multiprocessing import Pool

In [ ]:
train=pd.read_csv('../train.csv')
camp=pd.read_csv('../campaign_data.csv')
trans=pd.read_csv('../customer_transaction_data.csv')
cust=pd.read_csv('../customer_demographics.csv')
coupon=pd.read_csv('../coupon_item_mapping.csv')
item=pd.read_csv('../item_data.csv')
camp.start_date=pd.to_datetime(camp.start_date,format='%d/%m/%y')
camp.end_date=pd.to_datetime(camp.end_date,format='%d/%m/%y')
test=pd.read_csv('../test_QyjYwdj.csv')

In [ ]:
train.head()

In [ ]:
def camp_feats(df):
    df['camp_duration']=(df['end_date']-df['start_date']).dt.days
    df['campaign_start_day']=df['start_date'].dt.day
    df['campaign_start_dow']=df['start_date'].dt.dayofweek
    df['campaign_end_day']=df['end_date'].dt.day
    df['campaign_end_dow']=df['end_date'].dt.dayofweek
    df=df.sort_values('start_date').reset_index(drop=True)
    df['days_since_last_campaign_start']=(df.start_date-df.start_date.shift(1)).dt.days
    df['days_since_last_campaign_end']=(df.end_date-df.end_date.shift(1)).dt.days
    df['days_between_start_end_of_prev_campaign']=(df.start_date-df.end_date.shift(1)).dt.days
    return df

In [ ]:
camp=camp_feats(camp)

In [ ]:
train=train.merge(camp,on='campaign_id',how='left')
test=test.merge(camp,on='campaign_id',how='left')

In [ ]:
coupon=coupon.merge(item,on='item_id')

In [ ]:
coupon.head()

In [ ]:
coupon_item_counts=coupon.groupby('coupon_id')['item_id'].nunique().to_dict()
coupon_brand_counts=coupon.groupby('coupon_id')['brand'].nunique().to_dict()
coupon_category_counts=coupon.groupby('coupon_id')['category'].nunique().to_dict()
item_coupon_counts=coupon.groupby('item_id')['coupon_id'].nunique().to_dict()
brand_coupon_counts=coupon.groupby('brand')['coupon_id'].nunique().to_dict()
category_coupon_counts=coupon.groupby('category')['coupon_id'].nunique().to_dict()

In [ ]:
item['num_coupons_per_item']=item.item_id.map(item_coupon_counts)
item.num_coupons_per_item=item.num_coupons_per_item.fillna(0)
item['num_coupons_per_brand']=item.brand.map(brand_coupon_counts)
item.num_coupons_per_brand=item.num_coupons_per_brand.fillna(0)
item['num_coupons_per_category']=item.category.map(category_coupon_counts)
item.num_coupons_per_category=item.num_coupons_per_category.fillna(0)

In [ ]:
item['num_items_per_brand']=item.brand.map(item.groupby('brand')['item_id'].nunique().to_dict())
item['num_items_per_category']=item.category.map(item.groupby('category')['item_id'].nunique().to_dict())

In [ ]:
item.head()

In [ ]:
trans=trans.merge(item,on='item_id')

In [ ]:
trans['price_after_OD']=trans['selling_price']-trans['other_discount']
trans['cost_price']=trans['price_after_OD']-trans['coupon_discount']
trans['date']=pd.to_datetime(trans['date'])
trans.sort_values('date',inplace=True)
trans['coupon_redeemed']=(trans['coupon_discount']<0).astype('int')

In [ ]:
trans.head()

In [ ]:
trans['selling_price_ewm']=trans.groupby('customer_id')['selling_price'].apply(lambda x: x.ewm(halflife=2).mean())
trans['other_discount_ewm']=trans.groupby('customer_id')['other_discount'].apply(lambda x: x.ewm(halflife=2).mean())
trans['coupon_discount_ewm']=trans.groupby('customer_id')['coupon_discount'].apply(lambda x: x.ewm(halflife=2).mean())
trans['price_after_OD_ewm']=trans.groupby('customer_id')['price_after_OD'].apply(lambda x: x.ewm(halflife=2).mean())
trans['cost_price_ewm']=trans.groupby('customer_id')['cost_price'].apply(lambda x: x.ewm(halflife=2).mean())
trans['coupon_redeemed_ewm']=trans.groupby('customer_id')['coupon_redeemed'].apply(lambda x: x.ewm(halflife=2).mean())

In [ ]:
trans[trans.item_id==26830]

In [ ]:
def return_stats(x):
    return [x.mean(),x.sum(),x.std(),x.max()]

In [ ]:
df=pd.concat((train,test),axis=0)

In [ ]:
temp=df[['customer_id','start_date','coupon_id','id']]

In [ ]:
def get_feats(temp):
    return [temp.shape[0],temp.item_id.nunique(),temp.brand.nunique(),temp.category.nunique(),temp.item_id.mode()[0]]+\
return_stats(temp.quantity)+return_stats(temp.selling_price)+return_stats(temp.other_discount)+return_stats(temp['coupon_redeemed'])+\
return_stats(temp.coupon_discount)+return_stats(temp.num_coupons_per_brand)+\
return_stats(temp.num_coupons_per_category)+return_stats(temp.num_coupons_per_item)+return_stats(temp.cost_price)+\
return_stats(temp.selling_price_ewm)+return_stats(temp.other_discount_ewm)+return_stats(temp.coupon_discount_ewm)+\
return_stats(temp.price_after_OD_ewm)+return_stats(temp.cost_price_ewm)+return_stats(temp.coupon_redeemed_ewm)

In [ ]:
%%time
def get_feat(row):
    data_=row[1]
    temp=trans[(trans.customer_id==data_['customer_id'])&(trans.date<data_['start_date'])]
    
    if temp.shape[0]>0:
        feats=[data_.id]+get_feats(temp)
    else:
        feats=[data_.id]+np.repeat(0,65).tolist()
    return feats

try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()
cols=[(x+'_mean',x+'_sum',x+'_std',x+'_max') for x in
    ['qty','sp','od','coupon_redeemed','cd','num_coupons_per_brand','num_coupons_per_category','num_coupons_per_item','cost_price',
'selling_price_ewm','other_discount_ewm','coupon_discount_ewm','price_after_OD_ewm','cost_price_ewm','coupon_redeemed_ewm']]
cols=['id','trans_shape','unique_item','unique_brand','unique_category','item_id']+[x for t in cols for x in t]    
cust_hist=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [ ]:
cust_hist.shape

In [ ]:
train=train.merge(cust_hist,on='id')
test=test.merge(cust_hist,on='id')

In [ ]:
train.shape

In [ ]:
train=train.merge(cust,on='customer_id',how='left')
test=test.merge(cust,on='customer_id',how='left')

In [ ]:
train.shape,test.shape

In [ ]:
train.to_pickle('../train_v2.pkl')
test.to_pickle('../test_v2.pkl')